In [10]:
import argparse
from webcam_utils import realtime_emotions
from prediction_utils import prediction_path
from emotion_recommend import recommend_by_emotion
import threading
import logging
import time

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 


In [11]:
def __get_logger():
    __logger = logging.getLogger('logger')
    formatter = logging.Formatter(
        '%(levelname)s##%(asctime)s##%(message)s')
    fileHandler = logging.FileHandler('./log.txt')
    fileHandler.setFormatter(formatter)
    __logger.addHandler(fileHandler)
    __logger.setLevel(logging.DEBUG)

    return __logger

logger = __get_logger()

In [12]:
rest_api_host = 'localhost'
rest_api_port = 5000

tcp_socket_host = 'localhost'
tcp_socket_port = 6000

In [13]:
# TODO: Socekt list (Need socket num to distinguish)

class SocketList():
    def __init__(self):
        self.lock = threading.Lock()
        self.socket_list = None
    
    def is_empty(self):
        if self.socket_list is None:
            return True
        else:
            return False

    def send(self, data):
        self.lock.acquire()
        try:
            self.socket_list.send(data)
        finally:
            self.lock.release()
        
    def add_socket(self, socket):
        self.lock.acquire()
        try:
            self.socket_list = socket
        finally:
            self.lock.release()

In [14]:
# for running realtime emotion detection
import easydict

def emotion_detection(img_str):
    return prediction_path(img_str)
    

In [15]:
from flask import Flask, request, jsonify

def send_to_socket(data):
    global socket_list
    if not socket_list.is_empty():
        logger.info('write to socket %s', data)
        try:
            socket_list.send(data.encode("utf-8"))
        except Exception as e:
            logger.error(e)
    else:
        logger.info('No socket')

def rest_api_work():
    app = Flask(__name__)

    @app.route('/emotion', methods = ['POST'])
    def analysisEmotion():
        logger.info('Emotion Request')
        try:
            imageData = request.get_json()['image']
            emotion_request_json = jsonify({'analysis_result': emotion_detection(imageData)})
            send_to_socket(emotion_request_json.get_data(as_text=True))
            return emotion_request_json
        except Exception as e:
            logger.error(e)

    @app.route('/recommend', methods = ['POST'])
    def analysisEmotionAndReturnRecommends():
        logger.info('Recommendations Request')
        try:
            imageData = request.get_json()['image']
            path = request.get_json()['path']
            emotion = emotion_detection(imageData)
            recommended_list = recommend_by_emotion(emotion, path)

            emotion_request_json = jsonify({
                    'header': 'EMOTIONAL_STATE',
                    'analysis_result':emotion,
                    'recommended_list':recommended_list
                    })

            send_to_socket(emotion_request_json.get_data(as_text=True))
            return emotion_request_json
        except Exception as e:
            logger.error(e)

    @app.route('/wakeup', methods = ['GET'])
    def wakeup():
        logger.info('Wakeup Request')
        try:
            wakeup_request_json = jsonify({'status': 'open'}) 
            send_to_socket(wakeup_request_json.get_data(as_text=True))
            return wakeup_request_json
        except Exception as e:
            logger.error(e)

    app.run(host=rest_api_host, port=rest_api_port)

In [16]:
import socketserver
import socket

class MyTcpHandler(socketserver.BaseRequestHandler):
    def handle(self):
        try:
            logger.info('New socket added') 
            global socket_list
            socket_list.add_socket(self.request)

            while True:
                data = self.request.recv(1024)

                # 빈 문자열을 수신하면 루프를 중지합니다. 
                if not data:
                    break

                # 수신받은 문자열을 출력합니다.
                logger.info('Received from %s', data.decode())
            return
        except Exception as e:
            logger.error(e)

In [17]:
def socket_work():
    def runServer():
        try:
            logger.info('Server started')
            server = socketserver.TCPServer((tcp_socket_host,tcp_socket_port),MyTcpHandler)
            server.serve_forever()
        except KeyboardInterrupt:
            logger.info('Server ended')
    
    runServer()

In [18]:
global socket_list
socket_list = SocketList()

th1 = threading.Thread(target=rest_api_work)
#th2 = threading.Thread(target=socket_work)

th1.start()
#th2.start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://localhost:5000/ (Press CTRL+C to quit)
